In [1]:
import mysql.connector
import pandas as pd
from pandas import Index
import os

### There are two types of yelp data
data set can be download here 
https://www.yelp.com/dataset/download 

1. photos are stored in aws s3 bucket (lab account)
2. records are stored in aws relation database (temperately stored in wesley's aws account)

The purpose of the use of data is to make our lily more knowledgeable.  
Lily can make restaurant recommendation to the user and show some pictures of the resturant

###  SQL database Schema

This is the database schema defined
```sql
create table Business(
  business_id varchar(22) primary key,
  b_name text,
  neighborhood text,
  city text,
  state text,
  postal_code varchar(7),
  latitude float,
  longitude float,
  stars numeric,
  review_count int,
  is_open int
);

create table Photos(
  photo_id varchar(22),
  business_id varchar (22) references Business,
  label text,
  primary key (photo_id, business_id) 
);
```

database are stored in **my own aws account**, because I don't have permission to set up one in gamma lab. I would set up an aws rds as soon as I get the permission
```
          host     ="*************.amazonaws.com",
          user     ="************",
          passwd   ="**********",
          database ="**********"
```

Thre are total 1047 cities in the database, 'new york' is not in the database

## customized back end API call

####  the following ocde will be used in bot.py to customized action

In [10]:
class RestaurantAPI(object):
    def __init__(self):
        self.mydb = mysql.connector.connect(
          host     = os.environ['RDS_HOSTNAME'],
          user     = os.environ['RDS_USERNAME'],
          passwd   = os.environ['RDS_PASSWORD'],
          database = os.environ['RDS_DB_NAME'])
        
        self.cursor = self.mydb.cursor()

    def search_restaurant(self, city_name):
        query = "select b_name, business_id from Business where city = '{}' order by stars desc limit 10".format(city_name)
        self.cursor.execute(query)
        data = self.cursor.fetchall()
        return data


    def search_photo(self, restaurant_id):
        # return url or none
        query = "select photo_id from Photos where business_id = '{}' limit 5 ".format(restaurant_id)
        self.cursor.execute(query)
        photo_id_list = [record[0] for record in self.cursor.fetchall()]
        photo_url_list = ["https://s3.amazonaws.com/robot-advisor/{}.jpg".format(photo_id) for photo_id in photo_id_list]
        return photo_url_list

In [8]:
api=RestaurantAPI()

In [12]:
# if it match it will return a pandas table
api.search_restaurant("Las Vegas")

[('Dial Carpet Cleaning', '--WsruI0IGEoeRmkErU5Gg'),
 ('Pinnacle Restoration', '--Y7NhBKzLTbNliMUX_wfg'),
 ('Nacho Mobile Auto Repair', '--z7PM8AGaJP0aBmGMY7RA'),
 ('Leisure Lawn', '-0KSt9tXv6C015vmAcSIcg'),
 ('Bachelor Vegas', '-0VImVWDeqewaA3th8fzIg'),
 ('Pick N Puff', '-0d4L4U3vKnTT0MEgyX_WA'),
 ('Sugar Me Clean Beauty Lounge', '-0gEYa7iHAshDcMvJ0Vuag'),
 ('Red Rock Canyon Tours', '-1W8v_0LVdvXMtynNcn4aA'),
 ("Bavette's Steakhouse & Bar", '-1m9o3vGRA8IBPNvNqKLmA'),
 ('La Tapatia Market', '-1uVyyy3UXNA9WoPucNniA')]

In [46]:
restaurant_table=api.search_restaurant("Hudson")

In [49]:
list(restaurant_table['b_name'])

['Spot-On Home Improvements',
 'Peace Love & Little Donuts - Hudson',
 'Pure Barre - Hudson',
 'Cambridge Jewelers',
 'Kevin C Aiken, DDS',
 'Goldfish Swim School - Hudson',
 'Olive My Heart',
 'The Red Twig',
 'Falling Waters Spa',
 'Suburban Sit']

In [19]:
# if it match none, it will return None
api.search_restaurant("New York")

In [ ]:
# if it match a record, it will return empty list 

In [60]:
api.search_photo("-1m9o3vGRA8IBPNvNqKLmA")

['https://s3.amazonaws.com/robot-advisor/BC8FBTL_RP9lSwq6zjW3EQ.jpg',
 'https://s3.amazonaws.com/robot-advisor/drdIB75rz8KtWEVo63X5Mg.jpg',
 'https://s3.amazonaws.com/robot-advisor/EAgvjf-7mjSAZtcAQCwElA.jpg',
 'https://s3.amazonaws.com/robot-advisor/EISXbTioGIO1_TZcxp_4NQ.jpg',
 'https://s3.amazonaws.com/robot-advisor/el2VWIMbm8pdTR44Vql0vQ.jpg']

In [50]:
api.search_photo("8TapxIxtzS8OzIIMpdqWjw")

[]

In [22]:
api.search_photo("8TapxIxtzS8OzIIMpdqWjw")

[]

In [24]:
api.cursor.execute("select * from Photos limit 5")
for row in api.cursor:
    print(row)

('---SnSf4OfUFfJmCxw1DZA', 'XQ0RIDqgnkXueL6y1CY3Cg', 'food')
('--0uqWanwN31OkuuwJ1zjQ', 'XIg92ukZJn_1aiNx0OmusQ', 'food')
('--3gCyYz0Eo3r07tAvUbhg', 'VeiL_tgw7dsl-7IcnOsh0g', 'food')
('--3vR19cePIkGQBgcLsQkw', 'If6Bku2jkgPiikR6HBu-XQ', 'food')
('--5FyXiUudlvrJyUCvpZJA', 'aqrTv6QiU0J9Sxm_T--c5g', 'food')


In [54]:
api.cursor.execute("select * from Business where business_id = '8g4bxSb1MvOW0GZbj30nsg'")
for row in api.cursor:
    print(row)

('8g4bxSb1MvOW0GZbj30nsg', 'Falling Waters Spa', '', 'Hudson', 'OH', '44236', 41.2144, -81.444, Decimal('5'), 4, 1)


In [62]:
if api.search_photo('8g4bxSb1MvOW0GZbj30nsg'):
    print("hi")

In [42]:
# show some of the images

In [41]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://s3.amazonaws.com/robot-advisor/---SnSf4OfUFfJmCxw1DZA.jpg")

In [44]:
Image(url= "https://s3.amazonaws.com/robot-advisor/hqDoSctmKnrlwdwlez0rHQ.jpg")

In [45]:
Image(url= "https://s3.amazonaws.com/robot-advisor/NimlLnaxg0M49ULwenNylg.jpg")